In [49]:
import numpy as np
from sklearn.datasets import load_svmlight_file

def load_data(file, data_type):
    x, y = load_svmlight_file(file)
    with open(file) as f:
        if data_type == 0:
            qid = np.array([ int(line.split()[-1]) for line in f ])
        else:
            qid = np.array([ int(line.split()[1][4:]) for line in f ])
    y /= np.max(y)
    if (data_type < 2):
        order = np.argsort(qid)
        return x.toarray()[order], y[order], qid[order]
    return x.toarray(), y, qid    

In [7]:
def split(x, y, qid, train_ratio):
    n = qid.shape[0]
    train_inds = np.sort(np.random.choice(n, int(n * train_ratio), replace=False))
    test_inds = np.sort(np.setdiff1d(range(n), train_inds))
    return (
        x[train_inds], y[train_inds], qid[train_inds],
        x[test_inds], y[test_inds], qid[test_inds],
    )

In [50]:
x, y, qid = load_data("imat2009_learning.txt", 0)
train_x, train_y, train_qid, test_x, test_y, test_qid = split(x, y, qid, 0.8)
test_x, test_y, test_qid, val_x, val_y, val_qid = split(test_x, test_y, test_qid, 0.8)

In [9]:
from catboost import Pool
train = Pool(data=train_x, label=train_y, group_id=train_qid)
test = Pool(data=test_x, label=test_y, group_id=test_qid)
val = Pool(data=val_x, label=val_y, group_id=val_qid)

In [19]:
from catboost import CatBoost
from statistics import mean

parameters = { 'custom_metric': ['NDCG:top=20'], 'iterations': 2000, 'loss_function': 'PairLogitPairwise' }

model = CatBoost(parameters)
model.fit(train, eval_set=test)
print("train: ", mean(model.eval_metrics(train, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("test: ", mean(model.eval_metrics(test, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("val: ", mean(model.eval_metrics(val, ['NDCG:top=20'])['NDCG:top=20;type=Base']))

0:	learn: 0.6858619	test: 0.6873302	best: 0.6873302 (0)	total: 901ms	remaining: 30m
1:	learn: 0.6788133	test: 0.6815927	best: 0.6815927 (1)	total: 1.8s	remaining: 29m 58s
2:	learn: 0.6725527	test: 0.6765411	best: 0.6765411 (2)	total: 2.72s	remaining: 30m 13s
3:	learn: 0.6665212	test: 0.6711098	best: 0.6711098 (3)	total: 3.63s	remaining: 30m 11s
4:	learn: 0.6607893	test: 0.6669867	best: 0.6669867 (4)	total: 4.59s	remaining: 30m 33s
5:	learn: 0.6553799	test: 0.6624876	best: 0.6624876 (5)	total: 5.49s	remaining: 30m 25s
6:	learn: 0.6499746	test: 0.6580816	best: 0.6580816 (6)	total: 6.42s	remaining: 30m 28s
7:	learn: 0.6450880	test: 0.6543840	best: 0.6543840 (7)	total: 7.36s	remaining: 30m 33s
8:	learn: 0.6404242	test: 0.6507209	best: 0.6507209 (8)	total: 8.29s	remaining: 30m 33s
9:	learn: 0.6364233	test: 0.6476029	best: 0.6476029 (9)	total: 9.17s	remaining: 30m 25s
10:	learn: 0.6321391	test: 0.6438088	best: 0.6438088 (10)	total: 10.1s	remaining: 30m 22s
11:	learn: 0.6285598	test: 0.640534

93:	learn: 0.5110763	test: 0.5441704	best: 0.5441704 (93)	total: 1m 28s	remaining: 29m 59s
94:	learn: 0.5106253	test: 0.5439184	best: 0.5439184 (94)	total: 1m 29s	remaining: 29m 58s
95:	learn: 0.5100963	test: 0.5435063	best: 0.5435063 (95)	total: 1m 30s	remaining: 29m 58s
96:	learn: 0.5094842	test: 0.5431607	best: 0.5431607 (96)	total: 1m 31s	remaining: 29m 57s
97:	learn: 0.5087892	test: 0.5428222	best: 0.5428222 (97)	total: 1m 32s	remaining: 29m 56s
98:	learn: 0.5083339	test: 0.5425446	best: 0.5425446 (98)	total: 1m 33s	remaining: 29m 57s
99:	learn: 0.5078316	test: 0.5422602	best: 0.5422602 (99)	total: 1m 34s	remaining: 29m 56s
100:	learn: 0.5073398	test: 0.5418532	best: 0.5418532 (100)	total: 1m 35s	remaining: 29m 55s
101:	learn: 0.5068478	test: 0.5414505	best: 0.5414505 (101)	total: 1m 36s	remaining: 29m 56s
102:	learn: 0.5063390	test: 0.5410565	best: 0.5410565 (102)	total: 1m 37s	remaining: 29m 57s
103:	learn: 0.5057685	test: 0.5405925	best: 0.5405925 (103)	total: 1m 38s	remaining:

182:	learn: 0.4726962	test: 0.5199982	best: 0.5199982 (182)	total: 2m 56s	remaining: 29m 13s
183:	learn: 0.4724514	test: 0.5198188	best: 0.5198188 (183)	total: 2m 57s	remaining: 29m 11s
184:	learn: 0.4722332	test: 0.5196743	best: 0.5196743 (184)	total: 2m 58s	remaining: 29m 10s
185:	learn: 0.4719843	test: 0.5194897	best: 0.5194897 (185)	total: 2m 59s	remaining: 29m 9s
186:	learn: 0.4715985	test: 0.5193662	best: 0.5193662 (186)	total: 3m	remaining: 29m 9s
187:	learn: 0.4712043	test: 0.5191710	best: 0.5191710 (187)	total: 3m 1s	remaining: 29m 8s
188:	learn: 0.4708800	test: 0.5190270	best: 0.5190270 (188)	total: 3m 2s	remaining: 29m 7s
189:	learn: 0.4706157	test: 0.5188670	best: 0.5188670 (189)	total: 3m 3s	remaining: 29m 6s
190:	learn: 0.4703073	test: 0.5187968	best: 0.5187968 (190)	total: 3m 4s	remaining: 29m 5s
191:	learn: 0.4699749	test: 0.5187884	best: 0.5187884 (191)	total: 3m 5s	remaining: 29m 4s
192:	learn: 0.4696895	test: 0.5186371	best: 0.5186371 (192)	total: 3m 6s	remaining: 29

271:	learn: 0.4501388	test: 0.5076991	best: 0.5076991 (271)	total: 4m 20s	remaining: 27m 36s
272:	learn: 0.4499294	test: 0.5075987	best: 0.5075987 (272)	total: 4m 21s	remaining: 27m 34s
273:	learn: 0.4496965	test: 0.5075132	best: 0.5075132 (273)	total: 4m 22s	remaining: 27m 33s
274:	learn: 0.4495398	test: 0.5074858	best: 0.5074858 (274)	total: 4m 23s	remaining: 27m 32s
275:	learn: 0.4493435	test: 0.5074495	best: 0.5074495 (275)	total: 4m 24s	remaining: 27m 31s
276:	learn: 0.4492097	test: 0.5074040	best: 0.5074040 (276)	total: 4m 25s	remaining: 27m 30s
277:	learn: 0.4489477	test: 0.5073198	best: 0.5073198 (277)	total: 4m 26s	remaining: 27m 29s
278:	learn: 0.4487424	test: 0.5072096	best: 0.5072096 (278)	total: 4m 27s	remaining: 27m 28s
279:	learn: 0.4485511	test: 0.5071402	best: 0.5071402 (279)	total: 4m 28s	remaining: 27m 27s
280:	learn: 0.4483530	test: 0.5070800	best: 0.5070800 (280)	total: 4m 29s	remaining: 27m 25s
281:	learn: 0.4481807	test: 0.5069331	best: 0.5069331 (281)	total: 4m 

360:	learn: 0.4337383	test: 0.5002319	best: 0.5002319 (360)	total: 5m 42s	remaining: 25m 55s
361:	learn: 0.4335912	test: 0.5001753	best: 0.5001753 (361)	total: 5m 43s	remaining: 25m 54s
362:	learn: 0.4334283	test: 0.5000804	best: 0.5000804 (362)	total: 5m 44s	remaining: 25m 53s
363:	learn: 0.4332399	test: 0.4999400	best: 0.4999400 (363)	total: 5m 45s	remaining: 25m 52s
364:	learn: 0.4330594	test: 0.4997932	best: 0.4997932 (364)	total: 5m 46s	remaining: 25m 51s
365:	learn: 0.4329484	test: 0.4997624	best: 0.4997624 (365)	total: 5m 47s	remaining: 25m 50s
366:	learn: 0.4328123	test: 0.4997300	best: 0.4997300 (366)	total: 5m 48s	remaining: 25m 49s
367:	learn: 0.4326252	test: 0.4996807	best: 0.4996807 (367)	total: 5m 49s	remaining: 25m 47s
368:	learn: 0.4324949	test: 0.4996155	best: 0.4996155 (368)	total: 5m 49s	remaining: 25m 46s
369:	learn: 0.4323495	test: 0.4995435	best: 0.4995435 (369)	total: 5m 50s	remaining: 25m 45s
370:	learn: 0.4321990	test: 0.4994758	best: 0.4994758 (370)	total: 5m 

...

1584:	learn: 0.3245788	test: 0.4704397	best: 0.4704378 (1581)	total: 26m 16s	remaining: 6m 52s
1585:	learn: 0.3245164	test: 0.4703875	best: 0.4703875 (1585)	total: 26m 17s	remaining: 6m 51s
1586:	learn: 0.3244576	test: 0.4703849	best: 0.4703849 (1586)	total: 26m 18s	remaining: 6m 50s
1587:	learn: 0.3244029	test: 0.4703668	best: 0.4703668 (1587)	total: 26m 19s	remaining: 6m 49s
1588:	learn: 0.3243433	test: 0.4703296	best: 0.4703296 (1588)	total: 26m 19s	remaining: 6m 48s
1589:	learn: 0.3242938	test: 0.4703217	best: 0.4703217 (1589)	total: 26m 20s	remaining: 6m 47s
1590:	learn: 0.3242263	test: 0.4703303	best: 0.4703217 (1589)	total: 26m 21s	remaining: 6m 46s
1591:	learn: 0.3241662	test: 0.4703695	best: 0.4703217 (1589)	total: 26m 21s	remaining: 6m 45s
1592:	learn: 0.3240910	test: 0.4703134	best: 0.4703134 (1592)	total: 26m 22s	remaining: 6m 44s
1593:	learn: 0.3240282	test: 0.4702837	best: 0.4702837 (1593)	total: 26m 23s	remaining: 6m 43s
1594:	learn: 0.3239672	test: 0.4703221	best: 0.470

1671:	learn: 0.3194491	test: 0.4696010	best: 0.4695907 (1670)	total: 27m 18s	remaining: 5m 21s
1672:	learn: 0.3194032	test: 0.4696135	best: 0.4695907 (1670)	total: 27m 19s	remaining: 5m 20s
1673:	learn: 0.3193479	test: 0.4695892	best: 0.4695892 (1673)	total: 27m 20s	remaining: 5m 19s
1674:	learn: 0.3192914	test: 0.4695650	best: 0.4695650 (1674)	total: 27m 20s	remaining: 5m 18s
1675:	learn: 0.3192344	test: 0.4695674	best: 0.4695650 (1674)	total: 27m 21s	remaining: 5m 17s
1676:	learn: 0.3191746	test: 0.4695797	best: 0.4695650 (1674)	total: 27m 22s	remaining: 5m 16s
1677:	learn: 0.3191228	test: 0.4695790	best: 0.4695650 (1674)	total: 27m 23s	remaining: 5m 15s
1678:	learn: 0.3190649	test: 0.4695448	best: 0.4695448 (1678)	total: 27m 23s	remaining: 5m 14s
1679:	learn: 0.3190369	test: 0.4695039	best: 0.4695039 (1679)	total: 27m 24s	remaining: 5m 13s
1680:	learn: 0.3189869	test: 0.4694951	best: 0.4694951 (1680)	total: 27m 25s	remaining: 5m 12s
1681:	learn: 0.3189316	test: 0.4694940	best: 0.469

1758:	learn: 0.3145348	test: 0.4690668	best: 0.4690668 (1758)	total: 28m 54s	remaining: 3m 57s
1759:	learn: 0.3144812	test: 0.4690583	best: 0.4690583 (1759)	total: 28m 55s	remaining: 3m 56s
1760:	learn: 0.3144256	test: 0.4690449	best: 0.4690449 (1760)	total: 28m 56s	remaining: 3m 55s
1761:	learn: 0.3143682	test: 0.4690979	best: 0.4690449 (1760)	total: 28m 57s	remaining: 3m 54s
1762:	learn: 0.3143082	test: 0.4691072	best: 0.4690449 (1760)	total: 28m 57s	remaining: 3m 53s
1763:	learn: 0.3142645	test: 0.4690814	best: 0.4690449 (1760)	total: 28m 58s	remaining: 3m 52s
1764:	learn: 0.3141910	test: 0.4690631	best: 0.4690449 (1760)	total: 28m 59s	remaining: 3m 51s
1765:	learn: 0.3141239	test: 0.4691183	best: 0.4690449 (1760)	total: 29m	remaining: 3m 50s
1766:	learn: 0.3140667	test: 0.4691289	best: 0.4690449 (1760)	total: 29m 1s	remaining: 3m 49s
1767:	learn: 0.3140161	test: 0.4690930	best: 0.4690449 (1760)	total: 29m 1s	remaining: 3m 48s
1768:	learn: 0.3139530	test: 0.4690627	best: 0.4690449 (

1845:	learn: 0.3097871	test: 0.4682916	best: 0.4682782 (1832)	total: 29m 59s	remaining: 2m 30s
1846:	learn: 0.3097258	test: 0.4683205	best: 0.4682782 (1832)	total: 30m	remaining: 2m 29s
1847:	learn: 0.3096709	test: 0.4683387	best: 0.4682782 (1832)	total: 30m	remaining: 2m 28s
1848:	learn: 0.3096075	test: 0.4683493	best: 0.4682782 (1832)	total: 30m 1s	remaining: 2m 27s
1849:	learn: 0.3095585	test: 0.4683681	best: 0.4682782 (1832)	total: 30m 2s	remaining: 2m 26s
1850:	learn: 0.3095066	test: 0.4683603	best: 0.4682782 (1832)	total: 30m 3s	remaining: 2m 25s
1851:	learn: 0.3094643	test: 0.4683446	best: 0.4682782 (1832)	total: 30m 3s	remaining: 2m 24s
1852:	learn: 0.3094142	test: 0.4683561	best: 0.4682782 (1832)	total: 30m 4s	remaining: 2m 23s
1853:	learn: 0.3093708	test: 0.4683531	best: 0.4682782 (1832)	total: 30m 5s	remaining: 2m 22s
1854:	learn: 0.3093151	test: 0.4683263	best: 0.4682782 (1832)	total: 30m 6s	remaining: 2m 21s
1855:	learn: 0.3092621	test: 0.4683591	best: 0.4682782 (1832)	tot

1932:	learn: 0.3052458	test: 0.4679619	best: 0.4679090 (1916)	total: 31m 6s	remaining: 1m 4s
1933:	learn: 0.3051954	test: 0.4679600	best: 0.4679090 (1916)	total: 31m 6s	remaining: 1m 3s
1934:	learn: 0.3051458	test: 0.4679202	best: 0.4679090 (1916)	total: 31m 7s	remaining: 1m 2s
1935:	learn: 0.3050940	test: 0.4679116	best: 0.4679090 (1916)	total: 31m 8s	remaining: 1m 1s
1936:	learn: 0.3050419	test: 0.4678507	best: 0.4678507 (1936)	total: 31m 9s	remaining: 1m
1937:	learn: 0.3049840	test: 0.4678646	best: 0.4678507 (1936)	total: 31m 9s	remaining: 59.8s
1938:	learn: 0.3049405	test: 0.4678389	best: 0.4678389 (1938)	total: 31m 10s	remaining: 58.8s
1939:	learn: 0.3048860	test: 0.4678378	best: 0.4678378 (1939)	total: 31m 11s	remaining: 57.9s
1940:	learn: 0.3048317	test: 0.4677869	best: 0.4677869 (1940)	total: 31m 11s	remaining: 56.9s
1941:	learn: 0.3047749	test: 0.4678152	best: 0.4677869 (1940)	total: 31m 12s	remaining: 55.9s
1942:	learn: 0.3047140	test: 0.4677971	best: 0.4677869 (1940)	total: 

In [20]:
parameters = { 'custom_metric': ['NDCG:top=20'], 'iterations': 2000 }

model2 = CatBoost(parameters)
model2.fit(train, eval_set=test)
print("train: ", mean(model2.eval_metrics(train, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("test: ", mean(model2.eval_metrics(test, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("val: ", mean(model2.eval_metrics(val, ['NDCG:top=20'])['NDCG:top=20;type=Base']))

0:	learn: 0.9318349	test: 0.9281769	best: 0.9281769 (0)	total: 23.1ms	remaining: 46.2s
1:	learn: 0.9262258	test: 0.9228509	best: 0.9228509 (1)	total: 52.3ms	remaining: 52.3s
2:	learn: 0.9210393	test: 0.9178649	best: 0.9178649 (2)	total: 79.6ms	remaining: 53s
3:	learn: 0.9154819	test: 0.9126472	best: 0.9126472 (3)	total: 105ms	remaining: 52.2s
4:	learn: 0.9105854	test: 0.9081934	best: 0.9081934 (4)	total: 131ms	remaining: 52.3s
5:	learn: 0.9059748	test: 0.9039334	best: 0.9039334 (5)	total: 161ms	remaining: 53.6s
6:	learn: 0.9014484	test: 0.8997810	best: 0.8997810 (6)	total: 185ms	remaining: 52.6s
7:	learn: 0.8971164	test: 0.8956804	best: 0.8956804 (7)	total: 211ms	remaining: 52.4s
8:	learn: 0.8929015	test: 0.8918239	best: 0.8918239 (8)	total: 238ms	remaining: 52.7s
9:	learn: 0.8889131	test: 0.8881021	best: 0.8881021 (9)	total: 267ms	remaining: 53.2s
10:	learn: 0.8847584	test: 0.8843661	best: 0.8843661 (10)	total: 298ms	remaining: 53.9s
11:	learn: 0.8807125	test: 0.8806570	best: 0.880657

99:	learn: 0.7736770	test: 0.7830003	best: 0.7830003 (99)	total: 2.87s	remaining: 54.6s
100:	learn: 0.7731535	test: 0.7825274	best: 0.7825274 (100)	total: 2.91s	remaining: 54.7s
101:	learn: 0.7727878	test: 0.7822253	best: 0.7822253 (101)	total: 2.93s	remaining: 54.6s
102:	learn: 0.7723174	test: 0.7818344	best: 0.7818344 (102)	total: 2.96s	remaining: 54.5s
103:	learn: 0.7719264	test: 0.7814413	best: 0.7814413 (103)	total: 2.99s	remaining: 54.5s
104:	learn: 0.7715365	test: 0.7811038	best: 0.7811038 (104)	total: 3.01s	remaining: 54.4s
105:	learn: 0.7712115	test: 0.7808266	best: 0.7808266 (105)	total: 3.04s	remaining: 54.3s
106:	learn: 0.7709016	test: 0.7805798	best: 0.7805798 (106)	total: 3.07s	remaining: 54.3s
107:	learn: 0.7706215	test: 0.7803319	best: 0.7803319 (107)	total: 3.1s	remaining: 54.3s
108:	learn: 0.7702371	test: 0.7800035	best: 0.7800035 (108)	total: 3.13s	remaining: 54.2s
109:	learn: 0.7699951	test: 0.7798142	best: 0.7798142 (109)	total: 3.15s	remaining: 54.2s
110:	learn: 0

192:	learn: 0.7512321	test: 0.7636511	best: 0.7636511 (192)	total: 5.47s	remaining: 51.2s
193:	learn: 0.7510397	test: 0.7634905	best: 0.7634905 (193)	total: 5.5s	remaining: 51.2s
194:	learn: 0.7508337	test: 0.7633539	best: 0.7633539 (194)	total: 5.53s	remaining: 51.2s
195:	learn: 0.7506959	test: 0.7632410	best: 0.7632410 (195)	total: 5.55s	remaining: 51.1s
196:	learn: 0.7505572	test: 0.7631247	best: 0.7631247 (196)	total: 5.57s	remaining: 51s
197:	learn: 0.7503957	test: 0.7629482	best: 0.7629482 (197)	total: 5.6s	remaining: 51s
198:	learn: 0.7502619	test: 0.7628219	best: 0.7628219 (198)	total: 5.63s	remaining: 50.9s
199:	learn: 0.7501176	test: 0.7627221	best: 0.7627221 (199)	total: 5.66s	remaining: 50.9s
200:	learn: 0.7499397	test: 0.7625837	best: 0.7625837 (200)	total: 5.69s	remaining: 50.9s
201:	learn: 0.7497715	test: 0.7624441	best: 0.7624441 (201)	total: 5.71s	remaining: 50.9s
202:	learn: 0.7496018	test: 0.7622985	best: 0.7622985 (202)	total: 5.74s	remaining: 50.8s
203:	learn: 0.74

288:	learn: 0.7383878	test: 0.7530684	best: 0.7530684 (288)	total: 8.03s	remaining: 47.6s
289:	learn: 0.7382823	test: 0.7529726	best: 0.7529726 (289)	total: 8.06s	remaining: 47.5s
290:	learn: 0.7381450	test: 0.7528673	best: 0.7528673 (290)	total: 8.09s	remaining: 47.5s
291:	learn: 0.7380241	test: 0.7527836	best: 0.7527836 (291)	total: 8.11s	remaining: 47.5s
292:	learn: 0.7379233	test: 0.7526778	best: 0.7526778 (292)	total: 8.14s	remaining: 47.4s
293:	learn: 0.7377177	test: 0.7525091	best: 0.7525091 (293)	total: 8.17s	remaining: 47.4s
294:	learn: 0.7375886	test: 0.7524098	best: 0.7524098 (294)	total: 8.2s	remaining: 47.4s
295:	learn: 0.7374231	test: 0.7522690	best: 0.7522690 (295)	total: 8.22s	remaining: 47.4s
296:	learn: 0.7373422	test: 0.7522111	best: 0.7522111 (296)	total: 8.25s	remaining: 47.3s
297:	learn: 0.7372645	test: 0.7521593	best: 0.7521593 (297)	total: 8.28s	remaining: 47.3s
298:	learn: 0.7371449	test: 0.7520591	best: 0.7520591 (298)	total: 8.31s	remaining: 47.3s
299:	learn:

384:	learn: 0.7281635	test: 0.7451177	best: 0.7451177 (384)	total: 10.7s	remaining: 44.7s
385:	learn: 0.7280646	test: 0.7450461	best: 0.7450461 (385)	total: 10.7s	remaining: 44.7s
386:	learn: 0.7279215	test: 0.7449329	best: 0.7449329 (386)	total: 10.7s	remaining: 44.6s
387:	learn: 0.7278050	test: 0.7448504	best: 0.7448504 (387)	total: 10.7s	remaining: 44.6s
388:	learn: 0.7276747	test: 0.7447683	best: 0.7447683 (388)	total: 10.8s	remaining: 44.6s
389:	learn: 0.7276344	test: 0.7447495	best: 0.7447495 (389)	total: 10.8s	remaining: 44.6s
390:	learn: 0.7275233	test: 0.7446514	best: 0.7446514 (390)	total: 10.8s	remaining: 44.6s
391:	learn: 0.7274523	test: 0.7446099	best: 0.7446099 (391)	total: 10.9s	remaining: 44.5s
392:	learn: 0.7273866	test: 0.7445544	best: 0.7445544 (392)	total: 10.9s	remaining: 44.5s
393:	learn: 0.7273087	test: 0.7444866	best: 0.7444866 (393)	total: 10.9s	remaining: 44.4s
394:	learn: 0.7272583	test: 0.7444562	best: 0.7444562 (394)	total: 10.9s	remaining: 44.4s
395:	learn

...

1616:	learn: 0.6715137	test: 0.7176860	best: 0.7176860 (1616)	total: 43.9s	remaining: 10.4s
1617:	learn: 0.6714893	test: 0.7176804	best: 0.7176804 (1617)	total: 43.9s	remaining: 10.4s
1618:	learn: 0.6714642	test: 0.7176593	best: 0.7176593 (1618)	total: 43.9s	remaining: 10.3s
1619:	learn: 0.6714270	test: 0.7176465	best: 0.7176465 (1619)	total: 43.9s	remaining: 10.3s
1620:	learn: 0.6713847	test: 0.7176288	best: 0.7176288 (1620)	total: 44s	remaining: 10.3s
1621:	learn: 0.6713550	test: 0.7176168	best: 0.7176168 (1621)	total: 44s	remaining: 10.3s
1622:	learn: 0.6713265	test: 0.7176079	best: 0.7176079 (1622)	total: 44s	remaining: 10.2s
1623:	learn: 0.6713080	test: 0.7176056	best: 0.7176056 (1623)	total: 44s	remaining: 10.2s
1624:	learn: 0.6712805	test: 0.7175986	best: 0.7175986 (1624)	total: 44.1s	remaining: 10.2s
1625:	learn: 0.6712508	test: 0.7175894	best: 0.7175894 (1625)	total: 44.1s	remaining: 10.1s
1626:	learn: 0.6711989	test: 0.7175775	best: 0.7175775 (1626)	total: 44.1s	remaining: 10

1712:	learn: 0.6685866	test: 0.7167178	best: 0.7167178 (1712)	total: 46.6s	remaining: 7.81s
1713:	learn: 0.6685493	test: 0.7167127	best: 0.7167127 (1713)	total: 46.7s	remaining: 7.79s
1714:	learn: 0.6685272	test: 0.7167102	best: 0.7167102 (1714)	total: 46.7s	remaining: 7.76s
1715:	learn: 0.6685086	test: 0.7167089	best: 0.7167089 (1715)	total: 46.7s	remaining: 7.73s
1716:	learn: 0.6684800	test: 0.7167188	best: 0.7167089 (1715)	total: 46.8s	remaining: 7.71s
1717:	learn: 0.6684503	test: 0.7167060	best: 0.7167060 (1717)	total: 46.8s	remaining: 7.68s
1718:	learn: 0.6684280	test: 0.7167029	best: 0.7167029 (1718)	total: 46.8s	remaining: 7.65s
1719:	learn: 0.6683996	test: 0.7166882	best: 0.7166882 (1719)	total: 46.8s	remaining: 7.63s
1720:	learn: 0.6683661	test: 0.7166850	best: 0.7166850 (1720)	total: 46.9s	remaining: 7.6s
1721:	learn: 0.6683454	test: 0.7166795	best: 0.7166795 (1721)	total: 46.9s	remaining: 7.57s
1722:	learn: 0.6683055	test: 0.7166710	best: 0.7166710 (1722)	total: 46.9s	remain

1805:	learn: 0.6658049	test: 0.7159041	best: 0.7159041 (1805)	total: 49.2s	remaining: 5.29s
1806:	learn: 0.6657684	test: 0.7158992	best: 0.7158992 (1806)	total: 49.3s	remaining: 5.26s
1807:	learn: 0.6657424	test: 0.7158974	best: 0.7158974 (1807)	total: 49.3s	remaining: 5.23s
1808:	learn: 0.6657150	test: 0.7158970	best: 0.7158970 (1808)	total: 49.3s	remaining: 5.21s
1809:	learn: 0.6656950	test: 0.7158911	best: 0.7158911 (1809)	total: 49.3s	remaining: 5.18s
1810:	learn: 0.6656782	test: 0.7158895	best: 0.7158895 (1810)	total: 49.4s	remaining: 5.15s
1811:	learn: 0.6656432	test: 0.7158853	best: 0.7158853 (1811)	total: 49.4s	remaining: 5.12s
1812:	learn: 0.6656001	test: 0.7158635	best: 0.7158635 (1812)	total: 49.4s	remaining: 5.1s
1813:	learn: 0.6655792	test: 0.7158630	best: 0.7158630 (1813)	total: 49.4s	remaining: 5.07s
1814:	learn: 0.6655617	test: 0.7158584	best: 0.7158584 (1814)	total: 49.5s	remaining: 5.04s
1815:	learn: 0.6655117	test: 0.7158416	best: 0.7158416 (1815)	total: 49.5s	remain

1901:	learn: 0.6631659	test: 0.7152398	best: 0.7152398 (1901)	total: 51.8s	remaining: 2.67s
1902:	learn: 0.6631399	test: 0.7152307	best: 0.7152307 (1902)	total: 51.9s	remaining: 2.64s
1903:	learn: 0.6631133	test: 0.7152369	best: 0.7152307 (1902)	total: 51.9s	remaining: 2.62s
1904:	learn: 0.6630810	test: 0.7152244	best: 0.7152244 (1904)	total: 51.9s	remaining: 2.59s
1905:	learn: 0.6630515	test: 0.7152107	best: 0.7152107 (1905)	total: 51.9s	remaining: 2.56s
1906:	learn: 0.6630205	test: 0.7152062	best: 0.7152062 (1906)	total: 52s	remaining: 2.53s
1907:	learn: 0.6629757	test: 0.7151981	best: 0.7151981 (1907)	total: 52s	remaining: 2.51s
1908:	learn: 0.6629467	test: 0.7151939	best: 0.7151939 (1908)	total: 52s	remaining: 2.48s
1909:	learn: 0.6629073	test: 0.7151871	best: 0.7151871 (1909)	total: 52.1s	remaining: 2.45s
1910:	learn: 0.6628892	test: 0.7151903	best: 0.7151871 (1909)	total: 52.1s	remaining: 2.42s
1911:	learn: 0.6628536	test: 0.7151795	best: 0.7151795 (1911)	total: 52.1s	remaining: 

1997:	learn: 0.6604243	test: 0.7144940	best: 0.7144904 (1995)	total: 54.4s	remaining: 54.5ms
1998:	learn: 0.6603925	test: 0.7144864	best: 0.7144864 (1998)	total: 54.4s	remaining: 27.2ms
1999:	learn: 0.6603645	test: 0.7144795	best: 0.7144795 (1999)	total: 54.5s	remaining: 0us

bestTest = 0.7144795372
bestIteration = 1999

train:  0.7897306921692805
test:  0.6998561227092535
val:  0.6658570570575948


In [22]:
parameters = { 'custom_metric': ['NDCG:top=20'], 'iterations': 2000, 'loss_function': 'YetiRankPairwise' }

model3 = CatBoost(parameters)
model3.fit(train, eval_set=test)
print("train: ", mean(model3.eval_metrics(train, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("test: ", mean(model3.eval_metrics(test, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("val: ", mean(model3.eval_metrics(val, ['NDCG:top=20'])['NDCG:top=20;type=Base']))

0:	test: 0.6792013	best: 0.6792013 (0)	total: 324ms	remaining: 10m 48s
1:	test: 0.6839626	best: 0.6839626 (1)	total: 674ms	remaining: 11m 13s
2:	test: 0.6838652	best: 0.6839626 (1)	total: 1.05s	remaining: 11m 37s
3:	test: 0.6839049	best: 0.6839626 (1)	total: 1.43s	remaining: 11m 51s
4:	test: 0.6839333	best: 0.6839626 (1)	total: 1.8s	remaining: 11m 56s
5:	test: 0.6873550	best: 0.6873550 (5)	total: 2.17s	remaining: 12m
6:	test: 0.6874547	best: 0.6874547 (6)	total: 2.54s	remaining: 12m 3s
7:	test: 0.6880071	best: 0.6880071 (7)	total: 2.91s	remaining: 12m 5s
8:	test: 0.6881175	best: 0.6881175 (8)	total: 3.29s	remaining: 12m 8s
9:	test: 0.6887111	best: 0.6887111 (9)	total: 3.67s	remaining: 12m 9s
10:	test: 0.6895145	best: 0.6895145 (10)	total: 4.04s	remaining: 12m 10s
11:	test: 0.6895169	best: 0.6895169 (11)	total: 4.42s	remaining: 12m 11s
12:	test: 0.6890985	best: 0.6895169 (11)	total: 4.79s	remaining: 12m 11s
13:	test: 0.6891177	best: 0.6895169 (11)	total: 5.16s	remaining: 12m 11s
14:	tes

114:	test: 0.6948984	best: 0.6949726 (113)	total: 42.1s	remaining: 11m 30s
115:	test: 0.6948311	best: 0.6949726 (113)	total: 42.5s	remaining: 11m 30s
116:	test: 0.6949114	best: 0.6949726 (113)	total: 42.9s	remaining: 11m 30s
117:	test: 0.6949971	best: 0.6949971 (117)	total: 43.3s	remaining: 11m 29s
118:	test: 0.6950710	best: 0.6950710 (118)	total: 43.6s	remaining: 11m 29s
119:	test: 0.6950409	best: 0.6950710 (118)	total: 44s	remaining: 11m 29s
120:	test: 0.6949557	best: 0.6950710 (118)	total: 44.4s	remaining: 11m 29s
121:	test: 0.6949866	best: 0.6950710 (118)	total: 44.8s	remaining: 11m 28s
122:	test: 0.6950250	best: 0.6950710 (118)	total: 45.1s	remaining: 11m 28s
123:	test: 0.6951289	best: 0.6951289 (123)	total: 45.5s	remaining: 11m 28s
124:	test: 0.6951954	best: 0.6951954 (124)	total: 45.9s	remaining: 11m 27s
125:	test: 0.6951350	best: 0.6951954 (124)	total: 46.2s	remaining: 11m 27s
126:	test: 0.6951096	best: 0.6951954 (124)	total: 46.6s	remaining: 11m 27s
127:	test: 0.6950580	best: 

224:	test: 0.6965848	best: 0.6965860 (219)	total: 1m 22s	remaining: 10m 51s
225:	test: 0.6966404	best: 0.6966404 (225)	total: 1m 22s	remaining: 10m 51s
226:	test: 0.6966845	best: 0.6966845 (226)	total: 1m 23s	remaining: 10m 50s
227:	test: 0.6966117	best: 0.6966845 (226)	total: 1m 23s	remaining: 10m 50s
228:	test: 0.6966225	best: 0.6966845 (226)	total: 1m 24s	remaining: 10m 49s
229:	test: 0.6966651	best: 0.6966845 (226)	total: 1m 24s	remaining: 10m 49s
230:	test: 0.6966722	best: 0.6966845 (226)	total: 1m 24s	remaining: 10m 49s
231:	test: 0.6965828	best: 0.6966845 (226)	total: 1m 25s	remaining: 10m 49s
232:	test: 0.6966139	best: 0.6966845 (226)	total: 1m 25s	remaining: 10m 48s
233:	test: 0.6967845	best: 0.6967845 (233)	total: 1m 25s	remaining: 10m 48s
234:	test: 0.6968150	best: 0.6968150 (234)	total: 1m 26s	remaining: 10m 47s
235:	test: 0.6968491	best: 0.6968491 (235)	total: 1m 26s	remaining: 10m 47s
236:	test: 0.6969279	best: 0.6969279 (236)	total: 1m 27s	remaining: 10m 47s
237:	test: 0

332:	test: 0.6977133	best: 0.6977262 (330)	total: 2m 2s	remaining: 10m 12s
333:	test: 0.6977995	best: 0.6977995 (333)	total: 2m 2s	remaining: 10m 11s
334:	test: 0.6979263	best: 0.6979263 (334)	total: 2m 3s	remaining: 10m 11s
335:	test: 0.6977676	best: 0.6979263 (334)	total: 2m 3s	remaining: 10m 11s
336:	test: 0.6977513	best: 0.6979263 (334)	total: 2m 3s	remaining: 10m 10s
337:	test: 0.6977372	best: 0.6979263 (334)	total: 2m 4s	remaining: 10m 10s
338:	test: 0.6976978	best: 0.6979263 (334)	total: 2m 4s	remaining: 10m 10s
339:	test: 0.6977096	best: 0.6979263 (334)	total: 2m 4s	remaining: 10m 9s
340:	test: 0.6977554	best: 0.6979263 (334)	total: 2m 5s	remaining: 10m 9s
341:	test: 0.6978084	best: 0.6979263 (334)	total: 2m 5s	remaining: 10m 9s
342:	test: 0.6977241	best: 0.6979263 (334)	total: 2m 6s	remaining: 10m 8s
343:	test: 0.6976096	best: 0.6979263 (334)	total: 2m 6s	remaining: 10m 8s
344:	test: 0.6976685	best: 0.6979263 (334)	total: 2m 6s	remaining: 10m 8s
345:	test: 0.6977638	best: 0.69

442:	test: 0.6984161	best: 0.6985407 (407)	total: 2m 43s	remaining: 9m 33s
443:	test: 0.6984287	best: 0.6985407 (407)	total: 2m 43s	remaining: 9m 32s
444:	test: 0.6984842	best: 0.6985407 (407)	total: 2m 43s	remaining: 9m 32s
445:	test: 0.6984741	best: 0.6985407 (407)	total: 2m 44s	remaining: 9m 32s
446:	test: 0.6984395	best: 0.6985407 (407)	total: 2m 44s	remaining: 9m 31s
447:	test: 0.6984990	best: 0.6985407 (407)	total: 2m 44s	remaining: 9m 31s
448:	test: 0.6984334	best: 0.6985407 (407)	total: 2m 45s	remaining: 9m 30s
449:	test: 0.6984364	best: 0.6985407 (407)	total: 2m 45s	remaining: 9m 30s
450:	test: 0.6984353	best: 0.6985407 (407)	total: 2m 45s	remaining: 9m 30s
451:	test: 0.6984301	best: 0.6985407 (407)	total: 2m 46s	remaining: 9m 29s
452:	test: 0.6983771	best: 0.6985407 (407)	total: 2m 46s	remaining: 9m 29s
453:	test: 0.6983800	best: 0.6985407 (407)	total: 2m 47s	remaining: 9m 28s
454:	test: 0.6984272	best: 0.6985407 (407)	total: 2m 47s	remaining: 9m 28s
455:	test: 0.6984173	best

...

1533:	test: 0.7005261	best: 0.7007016 (1384)	total: 9m 23s	remaining: 2m 51s
1534:	test: 0.7005261	best: 0.7007016 (1384)	total: 9m 23s	remaining: 2m 50s
1535:	test: 0.7005246	best: 0.7007016 (1384)	total: 9m 23s	remaining: 2m 50s
1536:	test: 0.7005591	best: 0.7007016 (1384)	total: 9m 24s	remaining: 2m 49s
1537:	test: 0.7005642	best: 0.7007016 (1384)	total: 9m 24s	remaining: 2m 49s
1538:	test: 0.7005224	best: 0.7007016 (1384)	total: 9m 25s	remaining: 2m 49s
1539:	test: 0.7005507	best: 0.7007016 (1384)	total: 9m 25s	remaining: 2m 48s
1540:	test: 0.7005480	best: 0.7007016 (1384)	total: 9m 25s	remaining: 2m 48s
1541:	test: 0.7005458	best: 0.7007016 (1384)	total: 9m 26s	remaining: 2m 48s
1542:	test: 0.7005595	best: 0.7007016 (1384)	total: 9m 26s	remaining: 2m 47s
1543:	test: 0.7005377	best: 0.7007016 (1384)	total: 9m 26s	remaining: 2m 47s
1544:	test: 0.7005582	best: 0.7007016 (1384)	total: 9m 27s	remaining: 2m 47s
1545:	test: 0.7005637	best: 0.7007016 (1384)	total: 9m 27s	remaining: 2m 46s

1640:	test: 0.7004955	best: 0.7007016 (1384)	total: 10m 2s	remaining: 2m 11s
1641:	test: 0.7004982	best: 0.7007016 (1384)	total: 10m 2s	remaining: 2m 11s
1642:	test: 0.7004955	best: 0.7007016 (1384)	total: 10m 2s	remaining: 2m 11s
1643:	test: 0.7004955	best: 0.7007016 (1384)	total: 10m 3s	remaining: 2m 10s
1644:	test: 0.7004955	best: 0.7007016 (1384)	total: 10m 3s	remaining: 2m 10s
1645:	test: 0.7004905	best: 0.7007016 (1384)	total: 10m 4s	remaining: 2m 9s
1646:	test: 0.7004905	best: 0.7007016 (1384)	total: 10m 4s	remaining: 2m 9s
1647:	test: 0.7005110	best: 0.7007016 (1384)	total: 10m 4s	remaining: 2m 9s
1648:	test: 0.7004570	best: 0.7007016 (1384)	total: 10m 5s	remaining: 2m 8s
1649:	test: 0.7004380	best: 0.7007016 (1384)	total: 10m 5s	remaining: 2m 8s
1650:	test: 0.7004380	best: 0.7007016 (1384)	total: 10m 5s	remaining: 2m 8s
1651:	test: 0.7004380	best: 0.7007016 (1384)	total: 10m 6s	remaining: 2m 7s
1652:	test: 0.7004329	best: 0.7007016 (1384)	total: 10m 6s	remaining: 2m 7s
1653:	t

1746:	test: 0.7004905	best: 0.7007016 (1384)	total: 10m 40s	remaining: 1m 32s
1747:	test: 0.7004890	best: 0.7007016 (1384)	total: 10m 40s	remaining: 1m 32s
1748:	test: 0.7004890	best: 0.7007016 (1384)	total: 10m 41s	remaining: 1m 32s
1749:	test: 0.7004890	best: 0.7007016 (1384)	total: 10m 41s	remaining: 1m 31s
1750:	test: 0.7004890	best: 0.7007016 (1384)	total: 10m 42s	remaining: 1m 31s
1751:	test: 0.7004890	best: 0.7007016 (1384)	total: 10m 42s	remaining: 1m 30s
1752:	test: 0.7004559	best: 0.7007016 (1384)	total: 10m 42s	remaining: 1m 30s
1753:	test: 0.7004559	best: 0.7007016 (1384)	total: 10m 43s	remaining: 1m 30s
1754:	test: 0.7004559	best: 0.7007016 (1384)	total: 10m 43s	remaining: 1m 29s
1755:	test: 0.7005099	best: 0.7007016 (1384)	total: 10m 43s	remaining: 1m 29s
1756:	test: 0.7004559	best: 0.7007016 (1384)	total: 10m 44s	remaining: 1m 29s
1757:	test: 0.7004559	best: 0.7007016 (1384)	total: 10m 44s	remaining: 1m 28s
1758:	test: 0.7004624	best: 0.7007016 (1384)	total: 10m 44s	rema

1853:	test: 0.7003322	best: 0.7007016 (1384)	total: 11m 18s	remaining: 53.5s
1854:	test: 0.7003205	best: 0.7007016 (1384)	total: 11m 19s	remaining: 53.1s
1855:	test: 0.7003205	best: 0.7007016 (1384)	total: 11m 19s	remaining: 52.7s
1856:	test: 0.7003217	best: 0.7007016 (1384)	total: 11m 20s	remaining: 52.4s
1857:	test: 0.7003217	best: 0.7007016 (1384)	total: 11m 20s	remaining: 52s
1858:	test: 0.7003045	best: 0.7007016 (1384)	total: 11m 20s	remaining: 51.6s
1859:	test: 0.7003217	best: 0.7007016 (1384)	total: 11m 21s	remaining: 51.3s
1860:	test: 0.7003096	best: 0.7007016 (1384)	total: 11m 21s	remaining: 50.9s
1861:	test: 0.7003096	best: 0.7007016 (1384)	total: 11m 22s	remaining: 50.5s
1862:	test: 0.7003096	best: 0.7007016 (1384)	total: 11m 22s	remaining: 50.2s
1863:	test: 0.7003096	best: 0.7007016 (1384)	total: 11m 22s	remaining: 49.8s
1864:	test: 0.7003096	best: 0.7007016 (1384)	total: 11m 23s	remaining: 49.5s
1865:	test: 0.7003492	best: 0.7007016 (1384)	total: 11m 23s	remaining: 49.1s
1

1960:	test: 0.7006318	best: 0.7007016 (1384)	total: 11m 58s	remaining: 14.3s
1961:	test: 0.7006318	best: 0.7007016 (1384)	total: 11m 58s	remaining: 13.9s
1962:	test: 0.7006318	best: 0.7007016 (1384)	total: 11m 59s	remaining: 13.6s
1963:	test: 0.7006318	best: 0.7007016 (1384)	total: 11m 59s	remaining: 13.2s
1964:	test: 0.7006649	best: 0.7007016 (1384)	total: 11m 59s	remaining: 12.8s
1965:	test: 0.7006654	best: 0.7007016 (1384)	total: 12m	remaining: 12.5s
1966:	test: 0.7006397	best: 0.7007016 (1384)	total: 12m	remaining: 12.1s
1967:	test: 0.7006066	best: 0.7007016 (1384)	total: 12m 1s	remaining: 11.7s
1968:	test: 0.7006071	best: 0.7007016 (1384)	total: 12m 1s	remaining: 11.4s
1969:	test: 0.7006056	best: 0.7007016 (1384)	total: 12m 1s	remaining: 11s
1970:	test: 0.7006387	best: 0.7007016 (1384)	total: 12m 2s	remaining: 10.6s
1971:	test: 0.7006387	best: 0.7007016 (1384)	total: 12m 2s	remaining: 10.3s
1972:	test: 0.7006498	best: 0.7007016 (1384)	total: 12m 2s	remaining: 9.89s
1973:	test: 0.7

In [57]:
x, y, qid = load_data("task3_train.txt", 1)
train_x, train_y, train_qid, test_x, test_y, test_qid = split(x, y, qid, 0.8)

train = Pool(data=train_x, label=train_y, group_id=train_qid)
test = Pool(data=test_x, label=test_y, group_id=test_qid)

In [58]:
parameters = { 'custom_metric': ['NDCG:top=20'], 'iterations': 2000, 'loss_function': 'PairLogitPairwise' }

model = CatBoost(parameters)
model.fit(train, eval_set=test)
print("train: ", mean(model.eval_metrics(train, ['NDCG:top=20'])['NDCG:top=20;type=Base']))
print("test: ", mean(model.eval_metrics(test, ['NDCG:top=20'])['NDCG:top=20;type=Base']))

0:	learn: 0.6854830	test: 0.6866226	best: 0.6866226 (0)	total: 548ms	remaining: 18m 15s
1:	learn: 0.6778913	test: 0.6802006	best: 0.6802006 (1)	total: 1.16s	remaining: 19m 23s
2:	learn: 0.6713474	test: 0.6748014	best: 0.6748014 (2)	total: 2.16s	remaining: 23m 57s
3:	learn: 0.6655185	test: 0.6700065	best: 0.6700065 (3)	total: 3.29s	remaining: 27m 22s
4:	learn: 0.6592850	test: 0.6650774	best: 0.6650774 (4)	total: 4.36s	remaining: 28m 59s
5:	learn: 0.6536440	test: 0.6603560	best: 0.6603560 (5)	total: 5.45s	remaining: 30m 9s
6:	learn: 0.6480292	test: 0.6554091	best: 0.6554091 (6)	total: 6.45s	remaining: 30m 37s
7:	learn: 0.6430108	test: 0.6511635	best: 0.6511635 (7)	total: 7.42s	remaining: 30m 48s
8:	learn: 0.6382894	test: 0.6471888	best: 0.6471888 (8)	total: 8.37s	remaining: 30m 51s
9:	learn: 0.6335053	test: 0.6425510	best: 0.6425510 (9)	total: 9.3s	remaining: 30m 50s
10:	learn: 0.6290277	test: 0.6385235	best: 0.6385235 (10)	total: 10.2s	remaining: 30m 46s
11:	learn: 0.6251257	test: 0.635

92:	learn: 0.5008006	test: 0.5374014	best: 0.5374014 (92)	total: 1m 10s	remaining: 23m 55s
93:	learn: 0.5001931	test: 0.5368731	best: 0.5368731 (93)	total: 1m 10s	remaining: 23m 52s
94:	learn: 0.4995207	test: 0.5365924	best: 0.5365924 (94)	total: 1m 11s	remaining: 23m 49s
95:	learn: 0.4988991	test: 0.5361998	best: 0.5361998 (95)	total: 1m 11s	remaining: 23m 45s
96:	learn: 0.4981845	test: 0.5354361	best: 0.5354361 (96)	total: 1m 12s	remaining: 23m 41s
97:	learn: 0.4975715	test: 0.5348896	best: 0.5348896 (97)	total: 1m 13s	remaining: 23m 37s
98:	learn: 0.4970882	test: 0.5345669	best: 0.5345669 (98)	total: 1m 13s	remaining: 23m 33s
99:	learn: 0.4965617	test: 0.5341707	best: 0.5341707 (99)	total: 1m 14s	remaining: 23m 30s
100:	learn: 0.4960665	test: 0.5339720	best: 0.5339720 (100)	total: 1m 14s	remaining: 23m 26s
101:	learn: 0.4954365	test: 0.5337161	best: 0.5337161 (101)	total: 1m 15s	remaining: 23m 22s
102:	learn: 0.4948552	test: 0.5329439	best: 0.5329439 (102)	total: 1m 15s	remaining: 2

181:	learn: 0.4614091	test: 0.5152155	best: 0.5152155 (181)	total: 2m 7s	remaining: 21m 14s
182:	learn: 0.4611435	test: 0.5150692	best: 0.5150692 (182)	total: 2m 8s	remaining: 21m 13s
183:	learn: 0.4608359	test: 0.5149886	best: 0.5149886 (183)	total: 2m 8s	remaining: 21m 11s
184:	learn: 0.4605061	test: 0.5149521	best: 0.5149521 (184)	total: 2m 9s	remaining: 21m 10s
185:	learn: 0.4601604	test: 0.5147445	best: 0.5147445 (185)	total: 2m 10s	remaining: 21m 9s
186:	learn: 0.4598713	test: 0.5146992	best: 0.5146992 (186)	total: 2m 10s	remaining: 21m 8s
187:	learn: 0.4595320	test: 0.5144639	best: 0.5144639 (187)	total: 2m 11s	remaining: 21m 8s
188:	learn: 0.4592222	test: 0.5142799	best: 0.5142799 (188)	total: 2m 12s	remaining: 21m 7s
189:	learn: 0.4589356	test: 0.5141553	best: 0.5141553 (189)	total: 2m 13s	remaining: 21m 7s
190:	learn: 0.4586584	test: 0.5140522	best: 0.5140522 (190)	total: 2m 13s	remaining: 21m 7s
191:	learn: 0.4583499	test: 0.5139479	best: 0.5139479 (191)	total: 2m 14s	remain

270:	learn: 0.4384936	test: 0.5067786	best: 0.5067786 (270)	total: 3m 3s	remaining: 19m 29s
271:	learn: 0.4382642	test: 0.5066772	best: 0.5066772 (271)	total: 3m 3s	remaining: 19m 28s
272:	learn: 0.4380354	test: 0.5067105	best: 0.5066772 (271)	total: 3m 4s	remaining: 19m 27s
273:	learn: 0.4378444	test: 0.5066525	best: 0.5066525 (273)	total: 3m 5s	remaining: 19m 25s
274:	learn: 0.4375875	test: 0.5065232	best: 0.5065232 (274)	total: 3m 5s	remaining: 19m 24s
275:	learn: 0.4373188	test: 0.5062973	best: 0.5062973 (275)	total: 3m 6s	remaining: 19m 23s
276:	learn: 0.4371689	test: 0.5062314	best: 0.5062314 (276)	total: 3m 6s	remaining: 19m 22s
277:	learn: 0.4370339	test: 0.5062012	best: 0.5062012 (277)	total: 3m 7s	remaining: 19m 21s
278:	learn: 0.4367640	test: 0.5060730	best: 0.5060730 (278)	total: 3m 8s	remaining: 19m 20s
279:	learn: 0.4365713	test: 0.5060210	best: 0.5060210 (279)	total: 3m 8s	remaining: 19m 19s
280:	learn: 0.4363729	test: 0.5059843	best: 0.5059843 (280)	total: 3m 9s	remaini

359:	learn: 0.4211268	test: 0.5007630	best: 0.5007630 (359)	total: 3m 59s	remaining: 18m 9s
360:	learn: 0.4209371	test: 0.5006981	best: 0.5006981 (360)	total: 3m 59s	remaining: 18m 8s
361:	learn: 0.4207768	test: 0.5003950	best: 0.5003950 (361)	total: 4m	remaining: 18m 7s
362:	learn: 0.4206484	test: 0.5003395	best: 0.5003395 (362)	total: 4m	remaining: 18m 6s
363:	learn: 0.4204671	test: 0.5002216	best: 0.5002216 (363)	total: 4m 1s	remaining: 18m 5s
364:	learn: 0.4203021	test: 0.5002395	best: 0.5002216 (363)	total: 4m 2s	remaining: 18m 5s
365:	learn: 0.4201129	test: 0.5002136	best: 0.5002136 (365)	total: 4m 2s	remaining: 18m 4s
366:	learn: 0.4199183	test: 0.5001462	best: 0.5001462 (366)	total: 4m 3s	remaining: 18m 3s
367:	learn: 0.4197690	test: 0.5001466	best: 0.5001462 (366)	total: 4m 4s	remaining: 18m 2s
368:	learn: 0.4195757	test: 0.5000652	best: 0.5000652 (368)	total: 4m 4s	remaining: 18m 1s
369:	learn: 0.4194049	test: 0.5000765	best: 0.5000652 (368)	total: 4m 5s	remaining: 18m
370:	l

...

1590:	learn: 0.2983533	test: 0.4761365	best: 0.4760386 (1581)	total: 16m 59s	remaining: 4m 22s
1591:	learn: 0.2982826	test: 0.4761410	best: 0.4760386 (1581)	total: 17m	remaining: 4m 21s
1592:	learn: 0.2982127	test: 0.4761368	best: 0.4760386 (1581)	total: 17m	remaining: 4m 20s
1593:	learn: 0.2981389	test: 0.4761548	best: 0.4760386 (1581)	total: 17m 1s	remaining: 4m 20s
1594:	learn: 0.2980835	test: 0.4761069	best: 0.4760386 (1581)	total: 17m 1s	remaining: 4m 19s
1595:	learn: 0.2980118	test: 0.4760821	best: 0.4760386 (1581)	total: 17m 2s	remaining: 4m 18s
1596:	learn: 0.2979486	test: 0.4760850	best: 0.4760386 (1581)	total: 17m 3s	remaining: 4m 18s
1597:	learn: 0.2978885	test: 0.4760652	best: 0.4760386 (1581)	total: 17m 3s	remaining: 4m 17s
1598:	learn: 0.2978086	test: 0.4760712	best: 0.4760386 (1581)	total: 17m 4s	remaining: 4m 16s
1599:	learn: 0.2977252	test: 0.4760466	best: 0.4760386 (1581)	total: 17m 5s	remaining: 4m 16s
1600:	learn: 0.2976592	test: 0.4760173	best: 0.4760173 (1600)	tot

1677:	learn: 0.2926317	test: 0.4754037	best: 0.4754037 (1677)	total: 17m 53s	remaining: 3m 26s
1678:	learn: 0.2925594	test: 0.4754343	best: 0.4754037 (1677)	total: 17m 54s	remaining: 3m 25s
1679:	learn: 0.2924884	test: 0.4754137	best: 0.4754037 (1677)	total: 17m 54s	remaining: 3m 24s
1680:	learn: 0.2924215	test: 0.4754054	best: 0.4754037 (1677)	total: 17m 55s	remaining: 3m 24s
1681:	learn: 0.2923789	test: 0.4754105	best: 0.4754037 (1677)	total: 17m 56s	remaining: 3m 23s
1682:	learn: 0.2923158	test: 0.4754786	best: 0.4754037 (1677)	total: 17m 56s	remaining: 3m 22s
1683:	learn: 0.2922459	test: 0.4755214	best: 0.4754037 (1677)	total: 17m 57s	remaining: 3m 22s
1684:	learn: 0.2921860	test: 0.4755782	best: 0.4754037 (1677)	total: 17m 58s	remaining: 3m 21s
1685:	learn: 0.2921164	test: 0.4755094	best: 0.4754037 (1677)	total: 17m 58s	remaining: 3m 20s
1686:	learn: 0.2920534	test: 0.4755196	best: 0.4754037 (1677)	total: 17m 59s	remaining: 3m 20s
1687:	learn: 0.2920016	test: 0.4754940	best: 0.475

1764:	learn: 0.2871214	test: 0.4751851	best: 0.4751739 (1763)	total: 18m 48s	remaining: 2m 30s
1765:	learn: 0.2870583	test: 0.4752109	best: 0.4751739 (1763)	total: 18m 49s	remaining: 2m 29s
1766:	learn: 0.2870090	test: 0.4751245	best: 0.4751245 (1766)	total: 18m 49s	remaining: 2m 28s
1767:	learn: 0.2869402	test: 0.4750787	best: 0.4750787 (1767)	total: 18m 50s	remaining: 2m 28s
1768:	learn: 0.2868897	test: 0.4751201	best: 0.4750787 (1767)	total: 18m 51s	remaining: 2m 27s
1769:	learn: 0.2868308	test: 0.4751761	best: 0.4750787 (1767)	total: 18m 51s	remaining: 2m 27s
1770:	learn: 0.2867607	test: 0.4751606	best: 0.4750787 (1767)	total: 18m 52s	remaining: 2m 26s
1771:	learn: 0.2867057	test: 0.4751874	best: 0.4750787 (1767)	total: 18m 52s	remaining: 2m 25s
1772:	learn: 0.2866316	test: 0.4752036	best: 0.4750787 (1767)	total: 18m 53s	remaining: 2m 25s
1773:	learn: 0.2865775	test: 0.4752361	best: 0.4750787 (1767)	total: 18m 53s	remaining: 2m 24s
1774:	learn: 0.2865180	test: 0.4752717	best: 0.475

1851:	learn: 0.2817702	test: 0.4745257	best: 0.4743781 (1829)	total: 19m 42s	remaining: 1m 34s
1852:	learn: 0.2817056	test: 0.4745862	best: 0.4743781 (1829)	total: 19m 43s	remaining: 1m 33s
1853:	learn: 0.2816472	test: 0.4746009	best: 0.4743781 (1829)	total: 19m 44s	remaining: 1m 33s
1854:	learn: 0.2815795	test: 0.4746262	best: 0.4743781 (1829)	total: 19m 44s	remaining: 1m 32s
1855:	learn: 0.2815381	test: 0.4746161	best: 0.4743781 (1829)	total: 19m 45s	remaining: 1m 31s
1856:	learn: 0.2814898	test: 0.4746435	best: 0.4743781 (1829)	total: 19m 46s	remaining: 1m 31s
1857:	learn: 0.2814193	test: 0.4746290	best: 0.4743781 (1829)	total: 19m 46s	remaining: 1m 30s
1858:	learn: 0.2813559	test: 0.4745671	best: 0.4743781 (1829)	total: 19m 47s	remaining: 1m 30s
1859:	learn: 0.2813075	test: 0.4745583	best: 0.4743781 (1829)	total: 19m 48s	remaining: 1m 29s
1860:	learn: 0.2812345	test: 0.4746501	best: 0.4743781 (1829)	total: 19m 48s	remaining: 1m 28s
1861:	learn: 0.2811856	test: 0.4746780	best: 0.474

1939:	learn: 0.2766530	test: 0.4743193	best: 0.4743187 (1938)	total: 20m 39s	remaining: 38.3s
1940:	learn: 0.2766120	test: 0.4743175	best: 0.4743175 (1940)	total: 20m 40s	remaining: 37.7s
1941:	learn: 0.2765646	test: 0.4743493	best: 0.4743175 (1940)	total: 20m 41s	remaining: 37.1s
1942:	learn: 0.2765006	test: 0.4743669	best: 0.4743175 (1940)	total: 20m 41s	remaining: 36.4s
1943:	learn: 0.2764422	test: 0.4743330	best: 0.4743175 (1940)	total: 20m 42s	remaining: 35.8s
1944:	learn: 0.2763848	test: 0.4743594	best: 0.4743175 (1940)	total: 20m 42s	remaining: 35.1s
1945:	learn: 0.2763427	test: 0.4743522	best: 0.4743175 (1940)	total: 20m 43s	remaining: 34.5s
1946:	learn: 0.2763055	test: 0.4743889	best: 0.4743175 (1940)	total: 20m 44s	remaining: 33.9s
1947:	learn: 0.2762461	test: 0.4743497	best: 0.4743175 (1940)	total: 20m 44s	remaining: 33.2s
1948:	learn: 0.2761905	test: 0.4743702	best: 0.4743175 (1940)	total: 20m 45s	remaining: 32.6s
1949:	learn: 0.2761446	test: 0.4743716	best: 0.4743175 (1940

In [59]:
x, y, qid = load_data("task3_test.txt", 2)
predict = Pool(data=x, label=y, group_id=qid)

In [52]:
def save_data(file_in, file_out, labels):
    with open(file_out, "wt") as f_out, open(file_in) as f_in:
        for i, line in enumerate(f_in):
            f_out.write(str(labels[i]) + line[2:] + "\n")

In [60]:
predictions = model.predict(predict)
save_data("task3_test.txt", "task3_result.txt", predictions)